### Experiments: Comparing RSAST with SAST Ridge, HIVE-COTE, ROCKET, TS-CHIEF

In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
from cd_diagram.cd_function import *


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


In [2]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown']
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)
print(max_ds)


['Chinatown']
1


In [3]:
runs = 1
#list
not_found_ds =[]
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []


for ds in ds_sens:
    
    try:
        
        X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
        X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
        print("ds="+ds)
    except:
        print("not found ds="+ds)
        not_found_ds.append(ds)
        continue

    for i in range(runs):
        
        p=1
        k=10
        print("RSAST ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        start = time.time()
        random_state = None
        rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=k, len_method="both", classifier=RidgeClassifierCV())
        rsast_ridge.fit(X_train, y_train)
        end = time.time()
        list_score.append(rsast_ridge.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        list_rpoint.append(str(p))
        list_nb_per_class.append(str(k))
        list_method.append("ACF&PACF: Rsast")
        
        
        p=1
        k=10
        print("RSAST Max PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        start = time.time()
        random_state = None
        rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=k, len_method="Max PACF", classifier=RidgeClassifierCV())
        rsast_ridge.fit(X_train, y_train)
        end = time.time()
        list_score.append(rsast_ridge.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("Max PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        list_rpoint.append(str(p))
        list_nb_per_class.append(str(k))
        list_method.append("Max PACF: Rsast")
        
        print("ROCKET: kernels=10_000")
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("Rocket")
        '''
        min_shp_length = 3
        print("STC: min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(np.inf))
        
        max_shp_length = X_train.shape[1]
        start = time.time()
        #Creating a ShapeletTransform and fitting
        st = ShapeletTransform(min_shapelet_length=min_shp_length, max_shapelet_length=np.inf)
        X_train_sktime = from_2d_array_to_nested(pd.DataFrame(X_train))
        X_test_sktime = from_2d_array_to_nested(pd.DataFrame(X_test))
        st.fit(X_train_sktime, y_train)
        #Making the tranformation of time series and training a RandomForestClassifier model
        X_test_transformed = st.transform(X_test_sktime)
        X_train_transformed = st.transform(X_train_sktime)
        clf = RandomForestClassifier()
        clf.fit(X_train_transformed, y_train)
        
        end = time.time()
        list_score.append(clf.score(X_test_transformed, y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(max_shp_length))
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("STC")  
        '''
        '''
        min_shp_length = 3
        max_shp_length = X_train.shape[1]
        print("STC: min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(X_train.shape[1]))
        start = time.time()
        
        candidate_lengths = np.arange(min_shp_length, max_shp_length+1)
        random_state = None
        sast_ridge = SAST(cand_length_list=candidate_lengths,
                                nb_inst_per_class=1, 
                                random_state=random_state, classifier=RidgeClassifierCV())
        sast_ridge.fit(X_train, y_train)
        end = time.time()    
        list_score.append(sast_ridge.score(X_test, y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(max_shp_length))
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("SAST")  
        '''
        
 

df_result['accuracy']=list_score
df_result['time']=list_time
df_result['dataset_name']=list_dataset
df_result['hyperparameter']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['classifier_name']=list_method
df_result=pd.DataFrame(df_result)
# export a overall dataset with the comparison
df_result.to_csv("df_overall_comparison_results.csv")

ds=Chinatown
RSAST ACF&PACF: n_random_points=1 nb_inst_per_class=10
RSAST Max PACF: n_random_points=1 nb_inst_per_class=10
ROCKET: kernels=10_000


In [4]:
df_result=pd.read_csv("df_overall_comparison_results.csv")
df_result.columns

Index(['Unnamed: 0', 'accuracy', 'time', 'dataset_name', 'hyperparameter',
       'rpoint', 'nb_per_class', 'classifier_name'],
      dtype='object')

In [5]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['classifier_name','hyperparameter'],columns=['dataset_name'], aggfunc='mean')

,dataset_name,Chinatown
classifier_name,hyperparameter,
ACF&PACF: Rsast,ACF&PACF: n_random_points=1 nb_inst_per_class=10,0.935860
Max PACF: Rsast,Max PACF: n_random_points=1 nb_inst_per_class=10,0.909621
Rocket,num_kernels=10_000,0.982507


In [6]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['classifier_name','hyperparameter'],columns=['dataset_name'], aggfunc='var')


,dataset_name
classifier_name,hyperparameter


In [7]:
df_result

,Unnamed: 0,accuracy,time,dataset_name,hyperparameter,rpoint,nb_per_class,classifier_name
0,0,0.935860,3.461048,Chinatown,ACF&PACF: n_random_points=1 nb_inst_per_class=10,1,10,ACF&PACF: Rsast
1,1,0.909621,0.083317,Chinatown,Max PACF: n_random_points=1 nb_inst_per_class=10,1,10,Max PACF: Rsast
2,2,0.982507,0.364558,Chinatown,num_kernels=10_000,0,0,Rocket


In [8]:
# create a pivot table with the mean of time spent in minutes

pd.pivot_table(df_result, values='time', index=['classifier_name','hyperparameter'],columns=['dataset_name'], aggfunc='mean')

,dataset_name,Chinatown
classifier_name,hyperparameter,
ACF&PACF: Rsast,ACF&PACF: n_random_points=1 nb_inst_per_class=10,3.461048
Max PACF: Rsast,Max PACF: n_random_points=1 nb_inst_per_class=10,0.083317
Rocket,num_kernels=10_000,0.364558


In [ ]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, title="Overall comparison", name_file='cd-diagram_comparison_other_methods.png')
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)

In [13]:
average_ranks

,0
Max PACF: Rsast,3.0
ACF&PACF: Rsast,2.0
Rocket,1.0
